In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, MaxPooling3D
import numpy as np
import os
import cv2
import glob

Using TensorFlow backend.


In [2]:
train_dir = 'cnn_data_train'
train_path = os.path.join(train_dir, '*g')

train_imgsNom = glob.glob(train_path)
train_labels = [int(name[-5]) for name in train_imgsNom]
train_imgs = np.array([np.array(cv2.imread(img)) for img in train_imgsNom])

In [3]:
# val_dir = 'cnn_data_val'
# val_path = os.path.join(val_dir, '*g')

# val_imgs = glob.glob(val_path)
# val_labels = [int(name[-5]) for name in val_imgs]
# val_imgs = np.array([np.array(cv2.imread(img)) for img in val_imgs])

In [4]:
test_dir = 'cnn_data_test'
test_path = os.path.join(test_dir, '*g')

test_imgsNom = glob.glob(test_path)
test_labels = np.array([int(name[-5]) for name in test_imgsNom])
test_imgs = np.array([np.array(cv2.imread(img)) for img in test_imgsNom])

NameError: name 'train_label' is not defined

For image loading and preprocessing see:
https://keras.io/preprocessing/image/

In [6]:
batch_size = 128
num_classes = 2
epochs = 30

# input image dimensions
img_rows, img_cols = 60, 60

print('train_imgs shape:', train_imgs.shape)
print(train_imgs.shape[0], 'train samples')
print(test_imgs.shape[0], 'test samples')

# convert class vectors to binary class matrices
train_label = keras.utils.to_categorical(train_labels, num_classes)
# val_label = keras.utils.to_categorical(val_labels, num_classes)
test_label = keras.utils.to_categorical(test_labels, num_classes)

train_imgs shape: (1228, 60, 60, 3)
1228 train samples
216 test samples


#### The trashy model used by fronk:

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', 
                 input_shape=(img_rows,img_cols,3)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer = 'random_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer = 'random_normal'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

#### The model used by DNT:

In [8]:
model2 = Sequential()
model2.add(Conv2D(4, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3)))
model2.add(Conv2D(5, (3, 3), activation='relu', kernel_initializer = 'random_normal'))
model2.add(Dropout(0.3))
model2.add(Flatten())
model2.add(Dense(16, activation='relu', kernel_initializer = 'random_normal', input_shape=(24*24*5,)))
model2.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

### Our improvement of DNT

In [9]:
model3 = Sequential()
model3.add(Conv2D(4, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3)))
model3.add(Conv2D(5, (3, 3), activation='relu', kernel_initializer = 'random_normal'))
model3.add(MaxPooling2D(pool_size=(4,4), padding='valid'))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(16, activation='relu', kernel_initializer = 'random_normal', input_shape=(24*24*5,)))
model3.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

In [10]:
model3.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model3.fit(train_imgs, train_label,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=.2)
score = model3.evaluate(test_imgs, test_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 982 samples, validate on 246 samples
Epoch 1/30
982/982 [==============================] - 22s 23ms/step - loss: 1.4097 - acc: 0.4715 - val_loss: 0.8384 - val_acc: 0.6301
Epoch 2/30
982/982 [==============================] - 20s 20ms/step - loss: 0.7658 - acc: 0.6120 - val_loss: 0.6699 - val_acc: 0.6748
Epoch 3/30
982/982 [==============================] - 20s 20ms/step - loss: 0.6906 - acc: 0.6619 - val_loss: 0.6686 - val_acc: 0.6789
Epoch 4/30
982/982 [==============================] - 19s 20ms/step - loss: 0.5348 - acc: 0.7128 - val_loss: 0.5232 - val_acc: 0.6911
Epoch 5/30
982/982 [==============================] - 20s 20ms/step - loss: 0.4979 - acc: 0.7271 - val_loss: 0.4888 - val_acc: 0.7276
Epoch 6/30
982/982 [==============================] - 20s 20ms/step - loss: 0.4614 - acc: 0.7475 - val_loss: 0.4617 - val_acc: 0.7927
Epoch 7/30
982/982 [==============================] - 20s 20ms/step - loss: 0.4768 - acc: 0.7688 - val_loss: 0.4363 - val_acc: 0.8008
Epoch 8/30
982/9

In [14]:
from texttable import Texttable

pred_test =  model3.predict(test_imgs)
predT = np.array(pred_test[:,0]) < 0.5
realT = test_labels == 1
print(sum(predT == realT))
print(len(predT))
# print("accuracy is:",accuracy(predT,realT))
# print(predT)
# print(realT)
def accuracy(predT, realT):
    return sum(predT == realT)/ len(realT)    

def precision(predT,realT):
    # if it says robot is it robot?
    return sum(predT and realT)/predT

# def recall()

def evalMatrix(predT,realT):
    t = Texttable()
    t.add_rows([['Pred\Real', 'True', 'False '], ['True', sum(predT & realT), sum(predT & (~realT)) ], ['False',  sum(~predT & realT), sum(~ predT & ~ realT) ]])
    print(t.draw())
evalMatrix(np.array(predT),np.array(realT))
# print(~realT)

164
216
+-----------+------+--------+
| Pred\Real | True | False  |
+===========+======+========+
| True      | 57   | 50     |
+-----------+------+--------+
| False     | 2    | 107    |
+-----------+------+--------+


In [15]:
leon = np.array([True,True,False,False])
Hm = np.array([True,False,False,True])
print(leon & Hm)
print(165/216)

[ True False False False]
0.7638888888888888


In [ ]:
# model2.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# model2.fit(train_imgs, train_label,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_split=.2)
# score = model2.evaluate(test_imgs, test_label, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

The model used by B-Human

In [16]:
import matplotlib.pyplot as plt

# B-Human CNN komt hier

In [ ]:
np.array(test_imgsNom)[predT & (~realT)]
# print(test_imgsNom)

In [ ]:


print(test_labels)
plt.imshow(test_imgs[0])